## DIA 085: Exportar el Historial de Predicciones a CSV desde una App Flask

Hoy aprenderás a exportar el historial de predicciones guardado en SQLite a un archivo CSV desde tu aplicación web Flask. Esta funcionalidad es útil para reportes, auditoría o análisis en Excel/Pandas.

✅ ¿Qué aprenderás hoy?

📌 Leer registros desde SQLAlchemy.

📌 Convertir los datos a un archivo CSV con pandas.

📌 Permitir la descarga del archivo desde el navegador.

🗂️ 1. Estructura del Proyecto (igual que antes)
pgsql
Copiar
Editar
titanic_app/
├── app.py
├── model.pkl
├── db.sqlite3
├── templates/
│   ├── index.html
│   └── historial.html
🧩 2. Código Flask Actualizado (app.py)
Agregamos una nueva ruta /exportar que genera y descarga el CSV.

python
Copiar
Editar
from flask import Flask, render_template, request, send_file
from flask_sqlalchemy import SQLAlchemy
import joblib
import numpy as np
import datetime
import pandas as pd
import io

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

class Registro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    f1 = db.Column(db.Float)
    f2 = db.Column(db.Float)
    f3 = db.Column(db.Float)
    f4 = db.Column(db.Float)
    prediccion = db.Column(db.String(50))
    fecha = db.Column(db.DateTime, default=datetime.datetime.utcnow)

modelo = joblib.load("model.pkl")

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predecir", methods=["POST"])
def predecir():
    try:
        f1 = float(request.form["f1"])
        f2 = float(request.form["f2"])
        f3 = float(request.form["f3"])
        f4 = float(request.form["f4"])

        datos = np.array([[f1, f2, f3, f4]])
        pred = modelo.predict(datos)[0]

        nuevo_registro = Registro(f1=f1, f2=f2, f3=f3, f4=f4, prediccion=str(pred))
        db.session.add(nuevo_registro)
        db.session.commit()

        return render_template("index.html", resultado=f"✅ Resultado: Clase {pred}")
    except:
        return render_template("index.html", resultado="❌ Error en los datos")

@app.route("/historial")
def historial():
    registros = Registro.query.order_by(Registro.fecha.desc()).all()
    return render_template("historial.html", registros=registros)

@app.route("/exportar")
def exportar():
    registros = Registro.query.order_by(Registro.fecha.desc()).all()

    # Convertimos a DataFrame
    data = [{
        "ID": r.id,
        "Feature 1": r.f1,
        "Feature 2": r.f2,
        "Feature 3": r.f3,
        "Feature 4": r.f4,
        "Predicción": r.prediccion,
        "Fecha": r.fecha.strftime('%Y-%m-%d %H:%M:%S')
    } for r in registros]

    df = pd.DataFrame(data)

    # Guardamos en memoria
    output = io.StringIO()
    df.to_csv(output, index=False)
    output.seek(0)

    return send_file(
        io.BytesIO(output.getvalue().encode()),
        mimetype='text/csv',
        as_attachment=True,
        download_name='historial_predicciones.csv'
    )

if __name__ == "__main__":
    db.create_all()
    app.run(debug=True)
🖼️ 3. Actualizar historial.html para agregar botón de descarga
Agrega este botón arriba de la tabla:

html
Copiar
Editar
<a href="/exportar" class="btn btn-success mb-3">⬇️ Exportar a CSV</a>
🔍 Líneas Clave Explicadas
pandas.DataFrame(data): crea una tabla con los registros.

io.StringIO() y send_file(...): genera el archivo en memoria y lo envía al navegador.

download_name='historial_predicciones.csv': nombra el archivo a descargar.

